# Learning tensorflow

Importing the tensorflow library.

In [1]:
import tensorflow as tf

Checking the installed version of tensorflow:

In [2]:
print("Tensorflow version:", tf.__version__)

Tensorflow version: 2.17.0


### Load a dataset

Loand and prepare the MNIST dataset. The pixel values of the image ranges from 0 through 255. Scale these values to a range of 0 to 1 by dividing the value by `255.0`. This also converts the sample data from integers to floating-point numbers.

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [5]:
x_train, x_test = x_train / 255.0, x_test / 255.0

### Build a machine learning models

Build a `tf.keras.Sequential` model:

In [8]:

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

c:\Users\91930\Documents\github-deaxparadox\ArtOfAI\venv\aoai-tf\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Sequential is useful for stacking layers where each layer has one input tensor and one output tensor. Layers are functions with a known mathematical structure that can be reused and have trainable variables. Most TensorFlow models are composed of layers. This models uses the `Flatten`, `Dense` and `Dropout` layers.

For each example, the models returns a vectors of `logits` or `logodds` scores, one for each class.

In [10]:
predictions = model(x_train[:1]).numpy()
predictions


array([[-0.8182873 , -0.5454214 ,  0.39547503,  0.03994963,  0.3813517 ,
        -0.17519712,  0.107089  ,  0.97327995,  0.37274414,  0.5958315 ]],
      dtype=float32)

The `tf.nn.softmax` functions converts these logits to *probabilities* for each class.

In [11]:
tf.nn.softmax(predictions).numpy()

array([[0.03426404, 0.04501349, 0.11533705, 0.08082881, 0.11371955,
        0.06518222, 0.08644191, 0.20554474, 0.11274492, 0.14092325]],
      dtype=float32)

Define a loss function fortraining using `losses.SparseCategoricalCrossentropy`:

In [13]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

The loss function takes a vector of ground truth values and a vector of logits and returns a scalar loss for each example. This loss is equal to the negative log probability of the true class: This loss is zero if the model is sure of the correct class.

This untrained model gives probabitlies close to random (1/10 foreach class), so the intial loss should be class to `-tf.math.log(1/10) ~=2.3`.

In [14]:
loss_fn(y_train[:1], predictions).numpy()

2.7305686

Before starting training, configure and  compile the model using keras `Model.compile`. Set the `optimizer` class to `adam`, set the `loss` to the `loss_fn` function you defined earlier, and specify a metric to be valuated for the model by settings the metrics parameter to `accuracy`.

In [15]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

### Train and evaluate your model

Une the  `Model.fit` method to adjust your model parameters and minimize the loss:

In [16]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 987us/step - accuracy: 0.8614 - loss: 0.4781
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 931us/step - accuracy: 0.9558 - loss: 0.1502
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 923us/step - accuracy: 0.9682 - loss: 0.1070
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 934us/step - accuracy: 0.9726 - loss: 0.0871
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 954us/step - accuracy: 0.9767 - loss: 0.0748


The `Model.evaluate` method checks the models' performance, usually on a validation set or test set.

In [17]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 0s - 760us/step - accuracy: 0.9776 - loss: 0.0724


[0.07235181331634521, 0.9775999784469604]

The image classifier is now trained to ~98% accuracy on this dataset.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it.

In [19]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [21]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[5.9960612e-08, 7.9664816e-11, 9.1532269e-07, 2.7976477e-05,
        1.3738654e-12, 3.4389881e-08, 6.4073905e-15, 9.9996912e-01,
        1.4409436e-07, 1.8027563e-06],
       [3.4123016e-08, 2.1317921e-05, 9.9997604e-01, 1.6395809e-06,
        7.9592859e-16, 1.7540177e-07, 3.3019384e-07, 1.1647448e-13,
        4.5643338e-07, 1.0905891e-16],
       [5.3050851e-08, 9.9924421e-01, 1.3210956e-04, 1.6059190e-05,
        4.1384057e-05, 4.1831258e-06, 5.1493003e-06, 5.0000747e-04,
        5.6466768e-05, 5.1812566e-07],
       [9.9988413e-01, 2.2832609e-10, 6.4685672e-05, 5.2295348e-07,
        2.9257197e-07, 1.6419665e-06, 6.5722834e-06, 2.3941782e-05,
        8.4608580e-08, 1.8084065e-05],
       [5.4921402e-06, 1.3830893e-09, 1.7599550e-05, 4.8486942e-07,
        9.9712640e-01, 2.2310235e-07, 3.7635795e-07, 5.9340513e-05,
        7.8041621e-06, 2.7823232e-03]], dtype=float32)>

In [ ]:
porbability_model